# Exercício 02 - Comparação de Redes Neurais Rasas

Aluno: Frederico Luis de Azevedo

Professor: Dr. Francisco de Assis Boldt

## Bibliotecas e Inicialização

In [1]:
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

import numpy as np
import time

## Classificadores Utilizados

A comparação das redes neurais será feita utilizados os seguintes classificadores
- SVM com Kernel linear
- SGD com função de perda 'hinge' (SMV Linear com descida de gradiente em seu treinamento)

In [2]:
svm = SVC(kernel='linear')
sgd = SGDClassifier(loss='hinge')

In [3]:
clfs = [
    ('SGD',sgd),
    ('SVM',svm)
]

## Bases de dados

Para este exercício serão construídas as seguintes bases de dados binárias:
- Base 1: 500 registros com 20 características, balanceada.
- Base 2: 500 registros com 20 características, desbalanceada
- Base 3: 500 registros com 5 características, balanceada.
- Base 4: 500 registros com 5 características, desbalanceada
- Base 5: 10.000 registros com 20 características, balanceada.
- Base 6: 10.000 registros com 20 características, desbalanceada
- Base 7: 10.000 registros com 5 características, balanceada.
- Base 8: 10.000 registros com 5 características, desbalanceada

In [4]:
datasets = [
    ('Base 1', Bunch(n_samples=500, n_features=20, weights=[0.5, 0.5])),
    ('Base 2', Bunch(n_samples=500, n_features=20, weights=[0.1, 0.9])),
    ('Base 3', Bunch(n_samples=500, n_features=5, weights=[0.5, 0.5])),
    ('Base 4', Bunch(n_samples=500, n_features=5, weights=[0.1, 0.9])),
    ('Base 5', Bunch(n_samples=10000, n_features=20, weights=[0.5, 0.5])),
    ('Base 6', Bunch(n_samples=10000, n_features=20, weights=[0.1, 0.9])),
    ('Base 7', Bunch(n_samples=10000, n_features=5, weights=[0.5, 0.5])),
    ('Base 8', Bunch(n_samples=10000, n_features=5, weights=[0.1, 0.9]))
]

## Avaliação de Performance

A avaliação da performance das redes neurais será feita com uma versão modificada da classe PerformanceEvaluator criada na disciplina de Reconhecimento de Padrões.

A modificação foi feita para que a classe possa criar uma base de dados dinamicamente com base nos parâmetros: tamanho da base, quantidade de caracterísitcas e balanceamento.

In [5]:
class PerformanceEvaluator():
  def __init__(self, datasets, cross_val = False):
    self.datasets = datasets
    self.cross_val = cross_val
  def score(self, clf, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    clf.fit(X_train, y_train)
    return clf.score(X_test, y_test)
  def evaluate(self, clfs):
    for name, dataset in self.datasets:
        print('-------- {} --------'.format(name))
        print(' ')
        
        X, y = make_classification(n_samples=dataset.n_samples, n_features=dataset.n_features, weights=dataset.weights)
        
        #unique, counts = np.unique(y, return_counts=True)
        #print(dict(zip(unique, counts)))
        #print(X.shape)
        
        print('{:>25}: {}'.format('Samples', dataset.n_samples))
        print('{:>25}: {}'.format('Features', dataset.n_features))
        if (dataset.weights[1:] == dataset.weights[:-1]):
            print('{:>25}: {}'.format('Balanced', 'true'))
        else:
            print('{:>25}: {}'.format('Balanced', 'false'))
        
        print(' ')
        print('{:>25}'.format('---- eval ---'))
        print(' ')
        
        for clf_name, clf in clfs:
            start_time = time.time()
            if (self.cross_val):
                scores = cross_val_score(clf, X, y, cv=5)
                print('{:>25}: {}'.format(clf_name, scores))
                #print('{:>25}: {}'.format('Mean', scores.mean()))
                #print('{:>25}: {}'.format('Std Deviation', scores.std()))
                #print('{:>25}: {}'.format('Median', np.median(scores)))
                print('{:>25}: {}'.format('Best', scores.max()))
            else:
                print('{:>25}: {}'.format(clf_name, self.score(clf, X, y)))
            
            elapsed_time = time.time() - start_time
            print('{:>25}: {}'.format('Time Spent', time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
            print(' ')
       
        print(' ')

In [6]:
pe = PerformanceEvaluator(datasets, cross_val = True)
pe.evaluate(clfs)

-------- Base 1 --------
 
                  Samples: 500
                 Features: 20
                 Balanced: true
 
            ---- eval ---
 
                      SGD: [0.91089109 0.93       0.9        0.95       0.91919192]
                     Best: 0.95
               Time Spent: 00:00:00
 
                      SVM: [0.92079208 0.94       0.93       0.94       0.93939394]
                     Best: 0.94
               Time Spent: 00:00:00
 
 
-------- Base 2 --------
 
                  Samples: 500
                 Features: 20
                 Balanced: false
 
            ---- eval ---
 
                      SGD: [0.89108911 0.88118812 0.88       0.88888889 0.8989899 ]
                     Best: 0.898989898989899
               Time Spent: 00:00:00
 
                      SVM: [0.89108911 0.94059406 0.94       0.88888889 0.92929293]
                     Best: 0.9405940594059405
               Time Spent: 00:00:00
 
 
-------- Base 3 --------
 
                  Samples

## Conclusão

É possível observar nas bases menores, com 500 registros, que o SVM performa melhor independente da quantidade de características e do balanceamento. Apenas caso da Base 1 (500 registros, 20 características, balanceada) que o SGD performou melhor que o SVM mas por pouco. Em todo caso, é possível observar nos valores do cross_validation que os scores do SGD são mais esparsos. Caberia um teste para testar se mais folds ajudariam na performance do SGD.

Nas bases maiores, de 10.000 registros, ambos os classificadores performam com valores praticamente iguais sendo o melhor deles na última base (Base 8), com menos características e desbalanceada. Ou seja, a quantidade maior de registros, mesmo que aleatórios, fazem as duas redes ficarem com performances iguais entre si.

O melhor score para 500 registros também está na base com menos características e desbalanceada (Base 4), assim como na base de 10.000 registros está na Base 8, mostrando que uma configuração de menos características e desbalanceada é a melhor configuração para estas redes.